In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
hpi_yellow = '#f6a800'
hpi_orange = '#dd6108'
hpi_red = '#b1063a'
hpi_gray = '#5a6065'
hpi_blue = '#007a9e'
hpi_green = '#007f6f'
hpi_lime = '#779a0b'
hpi_purple = '#7664a0'

In [ ]:
def load_file(name):
    with open(name, 'r') as f:
        data = json.loads(f.read())
    return data

def plot_times(series: list[list[float]], labels: list[str] = []):
    parts = plt.violinplot(dataset=series, vert=False, showextrema=False, showmedians=True)
    plt.title("Client-side action processing times")
    plt.xlim(0)
    plt.xlabel('Per-action time in ms')

    plt.ylabel('Configuration')
    plt.yticks(
        ticks=range(1,len(series)+1),
        labels=labels)
    for pc in parts['bodies']:
        pc.set_alpha(1)
        pc.set_facecolor(hpi_orange)
        pc.set_edgecolor('k')
    
    parts['cmedians'].set_color(hpi_red)
    plt.show()

def load_series(fn):
    data = load_file(fn)
    plot_series = []
    plot_labels = []
    for session in data:
        settings = session['settings']
        for run in session['raw']:
            plot_series.append(run['times'])
            plot_labels.append(f'{settings["pas"]}P/{settings["ticks"]}T/{settings["run"]}R: {run["ctx"]["stage"]}')
    return plot_series, plot_labels

def plot_files(*fn):
    s,l = [], []
    for i, f in enumerate(fn):
        ds, dl = load_series(f)
        s += ds
        l += [f'{dll} ({i})' for dll in dl]
    plot_times(s, l)

In [ ]:
d = load_file('./../data/tick-patients/2023-06-08T14:25:57.929Z.json')
d += load_file('./../data/tick-patients/2023-06-08T14:41:29.940Z.json')
    
series = [run['raw']['[Exercise] Tick'] for run in d]
labels=[f'{run["settings"]["method"]}' for run in d]

plot_times(series, labels)

In [ ]:
plot_files('./../data/tick-patients/2023-06-12T08:34:31.639Z.json')